<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.5MB/s 
     |████████████████████████████████| 3.0MB 20.9MB/s 
     |████████████████████████████████| 890kB 46.1MB/s 


In [2]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer,BertConfig
import re
import numpy as np

<b>2. Load the vocabulary and ProtBert Model</b>

In [3]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )

In [4]:
model = TFBertModel.from_pretrained("Rostlab/prot_bert", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [5]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [6]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [7]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [8]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [9]:
embedding = model(input_ids)[0]

In [10]:
embedding = np.asarray(embedding)

In [11]:
attention_mask = np.asarray(attention_mask)

<b>6. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtBert model<b>

In [12]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [13]:
print(features)

[array([[ 0.09233958,  0.13914403, -0.05241528, ..., -0.13947995,
        -0.04280058,  0.07431365],
       [ 0.1150599 ,  0.01998255, -0.08627477, ..., -0.0094623 ,
        -0.18727243,  0.13169028],
       [ 0.04012017,  0.05467904, -0.04245842, ...,  0.02054806,
        -0.03819012,  0.10118145],
       ...,
       [ 0.08853209,  0.0228723 , -0.05121124, ..., -0.06733349,
        -0.03013721,  0.04913726],
       [ 0.10792378,  0.0977104 , -0.05832328, ..., -0.12765586,
        -0.06493296,  0.12894128],
       [ 0.05457975,  0.03635868, -0.0782121 , ..., -0.03016013,
        -0.06015305,  0.08903901]], dtype=float32), array([[ 0.08192439,  0.04021844,  0.00467392, ...,  0.08482227,
        -0.05522079,  0.02153459],
       [ 0.07777947,  0.0743171 ,  0.03168098, ...,  0.09620474,
        -0.02188272, -0.0145603 ],
       [ 0.13320833,  0.11049384, -0.05078609, ...,  0.09844306,
        -0.02032756,  0.05769319],
       [ 0.02869374,  0.07422507, -0.01506015, ..., -0.05882555,
     